Lo primero que realizo es importar las librerias que voy a utilizar en este proyecto

In [1]:
import scipy, scipy.integrate
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score


import seaborn as sns

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Se importa la data de las casas

In [2]:
data= pd.read_csv("train_precios_vivienda.csv")
data

C:\Users\Lenovo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (67,109,110,160,168,169,170,171,178,180) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id,fecha_aprobación,objeto,motivo,proposito,tipo_avaluo,tipo_credito,tipo_subsidio,departamento_inmueble,...,valor_area_construccion,area_otros,valor_area_otros,area_libre,valor_area_libre,valor_total_avaluo,valor_uvr,valor_avaluo_en_uvr,Longitud,Latitud
0,4112,5896,43090.624747,Remate,Remates,GarantÃ­a Hipotecaria,Remates,Vivienda,NaN,VALLE DEL CAUCA,...,"81968750,00",0,0,0,0,"145318750,00",2522304,"576134,95",0.000000,0.000000
1,7401,10570,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,QUINDÃO,...,0,157,78500000,No,0,713986654,257.23250000000002,2775647.14,-75.661152,4.544027
2,10223,14600,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,ANTIOQUIA,...,0,0,0,Si,0,270500000,259.4264,1042684.94,-75.584116,6.277020
3,4170,5967,43091.676139,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,VIS,CUNDINAMARCA,...,0,0,0,No,0,"84840000,00",252245,"336339,67",0.000000,0.000000
4,11073,15814,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,VIS,META,...,69306400,0,0,0,0,96346400,259.72770000000003,370951.58,-73.712370,3.565757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11566,11964,17047,NaN,OriginaciÃ³n,Leasing Habitacional,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,"BOGOTÃ, D. C.",...,0,0,0,No,0,709028000,259.57420000000002,2731504.13,-74.047845,4.711579
11567,5191,7419,43157.000000,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,ANTIOQUIA,...,0,0,0,Si,2193300,158356260,254.18279999999999,623001.48,-74.610458,10.463742
11568,5390,7687,NaN,OriginaciÃ³n,Leasing Habitacional,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,"BOGOTÃ, D. C.",...,0,0,0,No,0,572610000,254.4109,2250729.04,-74.109652,4.674083
11569,860,1247,42999.429115,Originación,Crédito hipotecario de vivienda,Garantía Hipotecaria,Hipotecario,Vivienda,NaN,"BOGOTÁ, D. C.",...,"97997000,00",0,0,0,0,"183290000,00",2517095,"728214,55",0.000000,0.000000


Visualizo como esta estructurada la información y cual es el tipo de cada caracteristica

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11571 entries, 0 to 11570
Columns: 222 entries, Unnamed: 0 to Latitud
dtypes: float64(6), int64(18), object(198)
memory usage: 19.6+ MB


In [4]:
data.dtypes

Unnamed: 0               int64
id                       int64
fecha_aprobación       float64
objeto                  object
motivo                  object
                        ...   
valor_total_avaluo      object
valor_uvr               object
valor_avaluo_en_uvr     object
Longitud               float64
Latitud                float64
Length: 222, dtype: object

De las 222 características que tiene el archivo selecciono las que pueden ser mas relevante en cuanto al precio del avaluo,
para este caso seleccione: tipo_avaluo', 'sector', 'alcantarillado_en_el_sector', 'acueducto_en_el_sector', 'gas_en_el_sector', 'energia_en_el_sector',   'estrato',  'demanda_interes',  'nivel_equipamiento_comercial', 'habitaciones', 'sala', 'bano_privado', 'total_cupos_parquedaro', 'numero_total_depositos', 'valor_total_avaluo', 'cocina', 'zona_de_ropas','Longitud', 'Latitud', 'area_privada', 'valor_area_privada' 

In [5]:
new_df = data[['tipo_avaluo', 'sector', 'alcantarillado_en_el_sector', 'acueducto_en_el_sector', 'gas_en_el_sector', 'energia_en_el_sector',   'estrato',  'demanda_interes',  'nivel_equipamiento_comercial', 'habitaciones', 'sala', 'bano_privado', 'total_cupos_parquedaro', 'numero_total_depositos', 'valor_total_avaluo', 'cocina', 'zona_de_ropas','Longitud', 'Latitud', 'area_privada', 'valor_area_privada' 

]].copy()



In [6]:
new_df.dtypes

tipo_avaluo                      object
sector                           object
alcantarillado_en_el_sector      object
acueducto_en_el_sector           object
gas_en_el_sector                 object
energia_en_el_sector             object
estrato                          object
demanda_interes                  object
nivel_equipamiento_comercial     object
habitaciones                     object
sala                             object
bano_privado                     object
total_cupos_parquedaro           object
numero_total_depositos           object
valor_total_avaluo               object
cocina                            int64
zona_de_ropas                     int64
Longitud                        float64
Latitud                         float64
area_privada                     object
valor_area_privada               object
dtype: object

Limpio los datos que son NaN 

In [7]:
new_dataset = new_df.dropna()
new_dataset

,tipo_avaluo,sector,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,estrato,demanda_interes,nivel_equipamiento_comercial,habitaciones,...,bano_privado,total_cupos_parquedaro,numero_total_depositos,valor_total_avaluo,cocina,zona_de_ropas,Longitud,Latitud,area_privada,valor_area_privada
0,Remates,Urbano,Si,Si,Si,Si,3,Media,Bueno,3,...,2,0,0,"145318750,00",1,1,0.000000,0.000000,0,0
1,Hipotecario,Urbano,Si,Si,Si,Si,5,Media,Regular Malo,4,...,2,0,0,713986654,1,1,-75.661152,4.544027,254.98,635486654
2,Hipotecario,Urbano,Si,Si,Si,Si,4,Media,Muy bueno,3,...,1,1,1,270500000,1,1,-75.584116,6.277020,82,246000000
3,Hipotecario,Urbano,Si,Si,Si,Si,2,Media,Muy bueno,3,...,1,0,0,"84840000,00",1,1,0.000000,0.000000,"48,48","84840000,00"
4,Hipotecario,Urbano,Si,Si,Si,Si,2,Media,Muy bueno,3,...,1,0,0,96346400,1,1,-73.712370,3.565757,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11566,Hipotecario,Urbano,Si,Si,Si,Si,5,Media,Bueno,3,...,1,2,1,709028000,1,1,-74.047845,4.711579,128.30000000000001,662028000
11567,Hipotecario,Urbano,Si,Si,Si,Si,3,Fuerte,Bueno,3,...,0,0,0,158356260,1,1,-74.610458,10.463742,64.08,156162960
11568,Hipotecario,Urbano,Si,Si,Si,Si,4,Media,Bueno,3,...,2,0,0,572610000,1,1,-74.109652,4.674083,190.87,572610000
11569,Hipotecario,Urbano,Si,Si,Si,Si,2,Débil,Bueno,4,...,0,0,0,"183290000,00",2,1,0.000000,0.000000,0,0


Se revisan una por una lo que contiene cada caracteristica del inmueble, se corrigen los errores que surgen por causa de tildes y ñ. Tambien se borran los datos que no correspondan a la categoria en la que se encuentra, por ejemplo en numero de habitaciones habian unos que contenian comentarios del inmueble.

In [8]:
print(new_dataset['area_privada'].value_counts())

0         3406
60          35
68          21
No          17
80          15
          ... 
87,94        1
105,45       1
709.21       1
44,09        1
105.96       1
Name: area_privada, Length: 5627, dtype: int64


In [9]:
new_dataset.drop(new_dataset[(new_dataset['tipo_avaluo'] == 'GarantÃ­a Hipotecaria')].index, inplace=True)

#Limpiamos datos de municipio inmueble
#new_dataset['municipio_inmueble'] = new_dataset['municipio_inmueble'].replace({'BOGOTÃ, D. C.': 'BOGOTÁ, D. C.'})

new_dataset.drop(new_dataset[(new_dataset['sector'] != 'Urbano') & (new_dataset['sector'] != 'Rural')].index, inplace=True)


new_dataset.drop(new_dataset[(new_dataset['alcantarillado_en_el_sector'] != 'Si') & (new_dataset['alcantarillado_en_el_sector'] != 'No')].index, inplace=True)

new_dataset.drop(new_dataset[(new_dataset['acueducto_en_el_sector'] != 'Si') & (new_dataset['acueducto_en_el_sector'] != 'No')].index, inplace=True)

new_dataset.drop(new_dataset[(new_dataset['gas_en_el_sector'] != 'Si') & (new_dataset['gas_en_el_sector'] != 'No')].index, inplace=True)

new_dataset.drop(new_dataset[(new_dataset['estrato'] == 'Industrial')].index, inplace=True)
new_dataset.drop(new_dataset[(new_dataset['estrato'] == 'Comercial')].index, inplace=True)



new_dataset['demanda_interes'] = new_dataset['demanda_interes'].replace({'DÃ©bil': 'Débil'})


new_dataset.drop(new_dataset[(new_dataset['habitaciones'] != '1') & (new_dataset['habitaciones'] != '2') & (new_dataset['habitaciones'] != '3') & (new_dataset['habitaciones'] != '4') & (new_dataset['habitaciones'] != '5')].index, inplace=True)


new_dataset.drop(new_dataset[(new_dataset['sala'] != '1') & (new_dataset['sala'] != '2') & (new_dataset['sala'] != '3') & (new_dataset['sala'] != '4') & (new_dataset['sala'] != '5')].index, inplace=True)

new_dataset['total_cupos_parquedaro'] = new_dataset['total_cupos_parquedaro'].replace({'No Tiene': '0'})
new_dataset.drop(new_dataset[(new_dataset['total_cupos_parquedaro'] == 'Exclusivo')].index, inplace=True)

new_dataset.drop(new_dataset[(new_dataset['numero_total_depositos'] == 'Privado')].index, inplace=True)







C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Mostramos como va quedando la data  y miramos cual es la categoria de cada una de ellas 

In [10]:
new_dataset

,tipo_avaluo,sector,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,estrato,demanda_interes,nivel_equipamiento_comercial,habitaciones,...,bano_privado,total_cupos_parquedaro,numero_total_depositos,valor_total_avaluo,cocina,zona_de_ropas,Longitud,Latitud,area_privada,valor_area_privada
0,Remates,Urbano,Si,Si,Si,Si,3,Media,Bueno,3,...,2,0,0,"145318750,00",1,1,0.000000,0.000000,0,0
1,Hipotecario,Urbano,Si,Si,Si,Si,5,Media,Regular Malo,4,...,2,0,0,713986654,1,1,-75.661152,4.544027,254.98,635486654
2,Hipotecario,Urbano,Si,Si,Si,Si,4,Media,Muy bueno,3,...,1,1,1,270500000,1,1,-75.584116,6.277020,82,246000000
3,Hipotecario,Urbano,Si,Si,Si,Si,2,Media,Muy bueno,3,...,1,0,0,"84840000,00",1,1,0.000000,0.000000,"48,48","84840000,00"
4,Hipotecario,Urbano,Si,Si,Si,Si,2,Media,Muy bueno,3,...,1,0,0,96346400,1,1,-73.712370,3.565757,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11566,Hipotecario,Urbano,Si,Si,Si,Si,5,Media,Bueno,3,...,1,2,1,709028000,1,1,-74.047845,4.711579,128.30000000000001,662028000
11567,Hipotecario,Urbano,Si,Si,Si,Si,3,Fuerte,Bueno,3,...,0,0,0,158356260,1,1,-74.610458,10.463742,64.08,156162960
11568,Hipotecario,Urbano,Si,Si,Si,Si,4,Media,Bueno,3,...,2,0,0,572610000,1,1,-74.109652,4.674083,190.87,572610000
11569,Hipotecario,Urbano,Si,Si,Si,Si,2,Débil,Bueno,4,...,0,0,0,"183290000,00",2,1,0.000000,0.000000,0,0


Las caracteristicas que son object, las que son descritas por numero las cambio a float y las que son de categoria las codifico usando dummies, que es una variable artificial creada para representar una caracteristica del inmueble con dos o mas categorias o niveles distintos.

En este caso codifico las siguientes 8 variables:
'tipo_avaluo', 'sector', 'alcantarillado_en_el_sector', 'acueducto_en_el_sector', 'gas_en_el_sector', 'energia_en_el_sector', 'demanda_interes', 'nivel_equipamiento_comercial'

In [11]:
#cambiamos las siguientes variables de object a float 
new_dataset['estrato']=new_dataset['estrato'].astype(float)
new_dataset['habitaciones']=new_dataset['habitaciones'].astype(float)
new_dataset['bano_privado']=new_dataset['bano_privado'].astype(float)
new_dataset['total_cupos_parquedaro']=new_dataset['total_cupos_parquedaro'].astype(float)
new_dataset['numero_total_depositos']=new_dataset['numero_total_depositos'].astype(float)
new_dataset['cocina']=new_dataset['cocina'].astype(float)
new_dataset['zona_de_ropas']=new_dataset['zona_de_ropas'].astype(float)
new_dataset['sala']=new_dataset['sala'].astype(float)




C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [12]:
#debido a que area privada, valor area privada y valor avaluo tienen unos datos que los decimales estan con (,) y otros por (.) se remplazan primero las (,) por (.) y luego se convierte en float
new_dataset['area_privada']=new_dataset['area_privada'].str.replace(',', '.').astype(float)
new_dataset['valor_area_privada']=new_dataset['valor_area_privada'].str.replace(',', '.').astype(float)
new_dataset['valor_total_avaluo']=new_dataset['valor_total_avaluo'].str.replace(',', '.').astype(float)


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [13]:
#se eliminan los datos en el que area privada es 0
new_dataset.drop(new_dataset[(new_dataset['valor_area_privada'] == 0)].index, inplace=True)


C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
s = (new_dataset.dtypes == 'object')
object_cols = list(s[s].index)
print("Variables categoricas:")
print(object_cols)
print('No. de variables categoricas: ',
      len(object_cols))


Variables categoricas:
['tipo_avaluo', 'sector', 'alcantarillado_en_el_sector', 'acueducto_en_el_sector', 'gas_en_el_sector', 'energia_en_el_sector', 'demanda_interes', 'nivel_equipamiento_comercial']
No. de variables categoricas:  8


# Codificación de variables categóricas mediante dummies
Esta es una variable artificial creada para representar un atributo con dos o más categorías/niveles distintos. 

In [15]:
# Se realiza la codificación de las variables categoricas escogidas y se agregan a la base de datos 
categorial_cols = ['tipo_avaluo', 'sector', 'alcantarillado_en_el_sector', 'acueducto_en_el_sector', 'gas_en_el_sector', 'energia_en_el_sector', 'demanda_interes', 'nivel_equipamiento_comercial']

for cc in categorial_cols:
    dummies = pd.get_dummies(new_dataset[cc], drop_first=False)
    dummies = dummies.add_prefix("{}#".format(cc))
    new_dataset.drop(cc, axis=1, inplace=True)
    new_dataset = new_dataset.join(dummies)






C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
new_dataset

,estrato,habitaciones,sala,bano_privado,total_cupos_parquedaro,numero_total_depositos,valor_total_avaluo,cocina,zona_de_ropas,Longitud,...,energia_en_el_sector#No,energia_en_el_sector#Si,demanda_interes#Débil,demanda_interes#Fuerte,demanda_interes#Media,demanda_interes#Nula,nivel_equipamiento_comercial#Bueno,nivel_equipamiento_comercial#En Proyecto,nivel_equipamiento_comercial#Muy bueno,nivel_equipamiento_comercial#Regular Malo
1,5.0,4.0,1.0,2.0,0.0,0.0,713986654.0,1.0,1.0,-75.661152,...,0,1,0,0,1,0,0,0,0,1
2,4.0,3.0,1.0,1.0,1.0,1.0,270500000.0,1.0,1.0,-75.584116,...,0,1,0,0,1,0,0,0,1,0
3,2.0,3.0,1.0,1.0,0.0,0.0,84840000.0,1.0,1.0,0.000000,...,0,1,0,0,1,0,0,0,1,0
5,3.0,3.0,1.0,1.0,0.0,0.0,194931000.0,1.0,1.0,-74.117696,...,0,1,0,0,1,0,1,0,0,0
7,3.0,3.0,1.0,0.0,0.0,0.0,85314600.0,1.0,1.0,-75.715128,...,0,1,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11563,5.0,2.0,1.0,1.0,2.0,1.0,832568000.0,1.0,1.0,-74.054740,...,0,1,0,0,1,0,1,0,0,0
11564,3.0,3.0,1.0,1.0,0.0,0.0,98062095.0,1.0,1.0,-73.058820,...,0,1,0,0,1,0,1,0,0,0
11566,5.0,3.0,1.0,1.0,2.0,1.0,709028000.0,1.0,1.0,-74.047845,...,0,1,0,0,1,0,1,0,0,0
11567,3.0,3.0,1.0,0.0,0.0,0.0,158356260.0,1.0,1.0,-74.610458,...,0,1,0,1,0,0,1,0,0,0


In [17]:
new_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7957 entries, 1 to 11568
Data columns (total 33 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   estrato                                    7957 non-null   float64
 1   habitaciones                               7957 non-null   float64
 2   sala                                       7957 non-null   float64
 3   bano_privado                               7957 non-null   float64
 4   total_cupos_parquedaro                     7957 non-null   float64
 5   numero_total_depositos                     7957 non-null   float64
 6   valor_total_avaluo                         7957 non-null   float64
 7   cocina                                     7957 non-null   float64
 8   zona_de_ropas                              7957 non-null   float64
 9   Longitud                                   7957 non-null   float64
 10  Latitud                

# Creacion de modelos para predecir valores de avaluo de nuevas viviendas

# Entrenamiento data
Primero escogemos X que va a ser toda la información menos lo que queremos que el modelo prediga y la Y va a ser la predicción
Luego se divide la data la que se usa como entrenamiento y la que se usa para validación

In [19]:
X = new_dataset.drop(['valor_total_avaluo'], axis=1)
Y = new_dataset['valor_total_avaluo']
 
# Dividir en entrenamiento y validación 
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X, Y, train_size=0.8, test_size=0.2, random_state=0)

In [20]:
def Mean_square_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.ones(len(y_true)), np.abs(y_true))))*100

# Se crea un modelo regresión lineal 
La regresión lineal predice el valor del avaluo en función de las características independientes dadas. 

In [21]:
model_LR = LinearRegression()
model_LR.fit(X_train, Y_train)
Y_pred = model_LR.predict(X_valid)
 
print(Mean_square_error(Y_valid, Y_pred))


848371835.5960336


# Modelo Random Forest (Bosque Aleatorio)
Un Random Forest es un conjunto de árboles de decisión combinados con bagging. Al usar bagging, lo que en realidad está pasando, es que distintos árboles ven distintas porciones de los datos. Ningún árbol ve todos los datos de entrenamiento. Esto hace que cada árbol se entrene con distintas muestras de datos para un mismo problema.

In [22]:
from sklearn.ensemble import RandomForestRegressor
 
model_RFR = RandomForestRegressor(n_estimators=10)
model_RFR.fit(X_train, Y_train)
Y_pred = model_RFR.predict(X_valid)
 
Mean_square_error(Y_valid, Y_pred)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


1079204.668367289

# Modelo Máquina de vectores de soporte (SVM)
Es un algoritmo de aprendizaje supervisado que se utiliza en problemas de clasificación y regresión

In [23]:
from sklearn import svm
from sklearn.svm import SVC

 
model_SVR = svm.SVR()
model_SVR.fit(X_train,Y_train)
Y_pred = model_SVR.predict(X_valid)
 
print(Mean_square_error(Y_valid, Y_pred))

37961456.071260445
